In [119]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


img = cv2.imread('lena.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imshow('original', img)
cv2.waitKey(0)
img = np.float32(img)

Encoding

In [120]:
dim = 512
QUANT_MATR = np.array([
    [16, 11, 10, 16, 24, 40, 51, 61],
    [12, 12, 14, 19, 26, 58, 60, 55],
    [14, 13, 16, 24, 40, 57, 69, 56],
    [14, 17, 22, 29, 51, 87, 80, 62],
    [18, 22, 37, 56, 68, 109, 103, 77],
    [24, 35, 55, 64, 81, 104, 113, 92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103, 99]
    ])

cur_block_i = 0
cur_block_j = 0
encoded = []

def update_encode():
    global prev
    global sym
    if q_dct[cur_i][cur_j] != prev:
        temp.append(sym)
        sym = [q_dct[cur_i][cur_j], 1]
    else:
        sym[1] += 1
    prev = q_dct[cur_i][cur_j]
   

while True:
    
    block = img[cur_block_i:cur_block_i + 8, cur_block_j:cur_block_j + 8]

    block = block - 128
    dct = cv2.dct(block)
    dct = np.round(dct)
    dct = np.int16(dct)
    q_dct = np.round(np.divide(dct,QUANT_MATR)) #quantization

    #run length encoding
    cur_i = 0
    cur_j = 0
    Direction = False # diagonal direction
    temp = [] # current block encoding

    # Process first element, q_dct[0][0]
    prev = q_dct[0][0] # previous processed element
    sym = [prev, 1] # symbol, [element, count]

    while True:
        
        if cur_i == 0 or cur_i == 7:
            Direction = not Direction
            cur_j += 1 #move right
            update_encode()
            if cur_i == cur_j == 7:
                temp.append(sym)
                break
        elif cur_j == 0 or cur_j == 7:
            Direction = not Direction
            cur_i += 1 #move down
            update_encode()

        if Direction == True: # move down left
            cur_i += 1
            cur_j -= 1
        else: #move up right
            cur_i -= 1
            cur_j += 1
        update_encode()

    encoded.append(temp)

    if cur_block_j + 8 < dim:
        cur_block_j += 8
    elif cur_block_i + 8 < dim:
        cur_block_i += 8
        cur_block_j = 0
    else:
        break
    
print(encoded)
print(np.shape(encoded))


[[[16.0, 1], [0.0, 1], [1.0, 1], [-0.0, 61]], [[15.0, 1], [2.0, 1], [1.0, 1], [-0.0, 61]], [[14.0, 1], [1.0, 2], [-0.0, 61]], [[13.0, 1], [0.0, 4], [1.0, 1], [-0.0, 58]], [[16.0, 1], [-2.0, 1], [1.0, 1], [-0.0, 2], [1.0, 1], [0.0, 58]], [[21.0, 1], [-2.0, 1], [-0.0, 3], [-1.0, 1], [0.0, 58]], [[20.0, 1], [3.0, 1], [0.0, 3], [-1.0, 1], [0.0, 58]], [[2.0, 1], [11.0, 1], [1.0, 1], [-0.0, 2], [-2.0, 1], [-1.0, 2], [-0.0, 5], [-1.0, 2], [0.0, 49]], [[-17.0, 1], [-0.0, 1], [1.0, 1], [-1.0, 1], [-0.0, 1], [2.0, 1], [0.0, 58]], [[-12.0, 1], [-2.0, 1], [1.0, 1], [-0.0, 1], [1.0, 1], [0.0, 59]], [[-11.0, 1], [-0.0, 1], [1.0, 1], [-0.0, 2], [1.0, 1], [0.0, 58]], [[-11.0, 1], [-0.0, 1], [1.0, 1], [-0.0, 61]], [[-11.0, 1], [0.0, 1], [1.0, 1], [-0.0, 61]], [[-7.0, 1], [-3.0, 1], [0.0, 62]], [[-3.0, 1], [-1.0, 2], [0.0, 2], [-1.0, 1], [-0.0, 58]], [[-1.0, 2], [1.0, 1], [-0.0, 2], [1.0, 1], [-0.0, 58]], [[1.0, 2], [0.0, 62]], [[2.0, 1], [1.0, 2], [-0.0, 1], [1.0, 1], [-0.0, 59]], [[1.0, 1], [-0.0, 63]

In [121]:
count = 0

for item in encoded:
    count += np.shape(item)[0]
count *= 2

print(count)

72946


Decoding

In [122]:
decoded = np.zeros((dim, dim))
cur_block_i = 0
cur_block_j = 0
cur_encoded_in = 0

def update_decode():
    global enc_block
    global temp
    if enc_block[0][1] == 0:
        enc_block = enc_block[1:]

    temp[cur_i][cur_j] = enc_block[0][0]
    enc_block[0][1] -= 1


while True:

    enc_block = np.array(encoded[cur_encoded_in])

    #run length decoding
    cur_i = 0
    cur_j = 0
    Direction = False # diagonal direction
    temp = np.zeros((8,8)) # current block decoding

    # Process first element
    temp[0][0] = enc_block[0][0]
    enc_block[0][1] -= 1
    
    while True:
        
        if cur_i == 0 or cur_i == 7:
            Direction = not Direction
            cur_j += 1 #move right
            update_decode()
            if cur_i == cur_j == 7:
                break
        elif cur_j == 0 or cur_j == 7:
            Direction = not Direction
            cur_i += 1 #move down
            update_decode()

        if Direction == True: # move down left
            cur_i += 1
            cur_j -= 1
        else: #move up right
            cur_i -= 1
            cur_j += 1
        update_decode()

    # dequantize
    temp = np.multiply(temp,QUANT_MATR)
    idct = cv2.idct(temp)
    idct = idct + 128
    decoded[cur_block_i:cur_block_i + 8, cur_block_j:cur_block_j + 8] = idct

    if cur_block_j + 8 < dim:
        cur_block_j += 8
    elif cur_block_i + 8 < dim:
        cur_block_i += 8
        cur_block_j = 0
    else:
        break

    cur_encoded_in += 1

print(decoded)

[[162.08055977 162.08055977 162.08055977 ... 177.03701404 157.98528662
  128.21227612]
 [161.7638134  161.7638134  161.7638134  ... 170.36694014 151.72939657
  123.91936728]
 [161.17854244 161.17854244 161.17854244 ... 169.82367597 152.2552229
  127.69904329]
 ...
 [ 45.28107407  46.52813348  48.41762212 ... 101.3383809   99.76012894
   98.26265762]
 [ 44.6958031   45.94286252  47.83235115 ... 103.39432536 103.70249635
  103.61470267]
 [ 44.37905674  45.62611615  47.51560479 ... 105.48660495 107.20445358
  108.15446177]]


In [123]:
decoded = np.round(decoded)
decoded = np.uint8(decoded)
print(decoded)



[[162 162 162 ... 177 158 128]
 [162 162 162 ... 170 152 124]
 [161 161 161 ... 170 152 128]
 ...
 [ 45  47  48 ... 101 100  98]
 [ 45  46  48 ... 103 104 104]
 [ 44  46  48 ... 105 107 108]]


In [124]:
cv2.imshow('decoded_image', decoded)
cv2.waitKey(0)

-1